In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r"C:\Users\Lenovo\ds\datasets\fake news.csv")

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df = df.dropna()

In [5]:
df.shape

(18285, 5)

In [6]:
x = df['title']
y = df['label']

In [7]:
import tensorflow as tf

In [9]:
tf.__version__

'2.16.1'

In [10]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [11]:
# Vocabulary size
voc_size = 5000

## One hot representation

In [12]:
messages = x.copy()

In [23]:
messages = messages.reset_index()

In [28]:
messages['title'][6]

'Benoît Hamon Wins French Socialist Party’s Presidential Nomination - The New York Times'

In [16]:
import nltk
import re
from nltk.corpus import stopwords  

In [17]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [30]:
# Data Preprocessing

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

corpus = []

for i in range(len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [32]:
corpus[0]

'hous dem aid even see comey letter jason chaffetz tweet'

In [34]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]

In [35]:
onehot_repr[0]

[1054, 3180, 3902, 3696, 2513, 3391, 4236, 2210, 4158, 2345]

## Embedding Representation

In [36]:
sent_len = 20
embedded_docs = pad_sequences(onehot_repr, padding = 'pre', maxlen= sent_len)
embedded_docs

array([[   0,    0,    0, ..., 2210, 4158, 2345],
       [   0,    0,    0, ..., 3656, 1845, 4240],
       [   0,    0,    0, ...,  124, 2828, 2355],
       ...,
       [   0,    0,    0, ..., 4505, 3563, 1199],
       [   0,    0,    0, ..., 1728, 4014, 4980],
       [   0,    0,    0, ..., 3404, 2012, 1944]])

## Creating LSTM Model

In [48]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Ensure voc_size and sent_len are defined and replace with actual values if needed
voc_size = 5000  # Example vocabulary size
sent_len = 20    # Example sentence length

embedding_vector_features = 40
model = Sequential()
model.add(Embedding(input_dim=voc_size, output_dim=embedding_vector_features, input_shape=(sent_len,)))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_10 (Embedding)             │ (None, 20, 40)              │         200,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 100)                 │          56,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 256,501 (1001.96 KB)

 Trainable params: 256,501 (1001.96 KB)

 Non-trainable params: 0 (0.00 B)

In [49]:
len(embedded_docs),y.shape

(18285, (18285,))

In [51]:
import numpy as np

X_final=np.array(embedded_docs)
y_final=np.array(y)

In [52]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [53]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_final, y_final, test_size = 0.3, random_state = 42)

In [55]:
model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 11s 31ms/step - accuracy: 0.7865 - loss: 0.4138 - val_accuracy: 0.9167 - val_loss: 0.1946
Epoch 2/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - accuracy: 0.9496 - loss: 0.1353 - val_accuracy: 0.9182 - val_loss: 0.1982
Epoch 3/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.9665 - loss: 0.0933 - val_accuracy: 0.9125 - val_loss: 0.2165
Epoch 4/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.9784 - loss: 0.0644 - val_accuracy: 0.9136 - val_loss: 0.2687
Epoch 5/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.9841 - loss: 0.0476 - val_accuracy: 0.9092 - val_loss: 0.2936
Epoch 6/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - accuracy: 0.9905 - loss: 0.0313 - val_accuracy: 0.9074 - val_loss: 0.3790
Epoch 7/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.9952 - loss: 0.0186 - val_accuracy: 0.9074 - val_loss: 0.3829
Epoch 8/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.9975 - loss: 0.0123 - val_acc

In [56]:
from tensorflow.keras.layers import Dropout

In [57]:
embedding_vector_features = 40
sent_len = 20

m2 = Sequential()
m2.add(Embedding(voc_size,embedding_vector_features, input_shape = (sent_len ,) ))
m2.add(Dropout(0.3))
m2.add(LSTM(100))
m2.add(Dropout(0.3))
m2.add(Dense(1,activation='sigmoid'))
m2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
m2.summary()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_11 (Embedding)             │ (None, 20, 40)              │         200,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 20, 40)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 100)                 │          56,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 256,501 (1001.96 KB)

 Trainable params: 256,501 (1001.96 KB)

 Non-trainable params: 0 (0.00 B)

In [62]:
y_pred_prob=model.predict(x_test)

172/172 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step


In [63]:
y_pred = (y_pred_prob > 0.5).astype("int32")


In [64]:
y_pred

array([[1],
       [0],
       [0],
       ...,
       [1],
       [0],
       [1]])

In [66]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9079475027342326